In [2]:
import pandas as pd

# Load data
df = pd.read_csv("/content/CSCE_Dashboard_Data.csv")
df['ticket_date'] = pd.to_datetime(df['ticket_date'])

# Escalation rate
escalation_rate = (df['status'] == 'Escalated').mean() * 100

# Tickets by channel
tickets_by_channel = df['channel'].value_counts()

# Avg resolution time per channel
avg_resolution = df.groupby('channel')['resolution_time'].mean()

# Agent performance
agent_summary = df.groupby('agent_id').agg({
    'ticket_id': 'count',
    'resolution_time': 'mean',
    'satisfaction_score': 'mean',
    'status': lambda x: (x == 'Escalated').sum()
}).rename(columns={
    'ticket_id': 'ticket_count',
    'resolution_time': 'avg_resolution',
    'satisfaction_score': 'avg_satisfaction',
    'status': 'escalated_tickets'
})

# Export for Power BI
agent_summary.to_excel("agent_summary.xlsx")

# Print results
print("Escalation Rate:", round(escalation_rate, 2), "%")
print("\nTickets by Channel:\n", tickets_by_channel)
print("\nAverage Resolution Time:\n", avg_resolution)
print("\nAgent Summary:\n", agent_summary)


Escalation Rate: 25.0 %

Tickets by Channel:
 channel
Email    3
Chat     3
Phone    2
Name: count, dtype: int64

Average Resolution Time:
 channel
Chat     50.000000
Email    68.333333
Phone    25.000000
Name: resolution_time, dtype: float64

Agent Summary:
           ticket_count  avg_resolution  avg_satisfaction  escalated_tickets
agent_id                                                                   
A1                   3       68.333333          2.333333                  1
A2                   3       50.000000          3.666667                  1
A3                   2       25.000000          5.000000                  0


**Simulating DB with SQLite**

**Escalation Rate by Channel**

In [9]:
import pandas as pd
import sqlite3

# Step 1: Load the dataset
df = pd.read_csv("/content/CSCE_Dashboard_Data.csv")

# Step 2: Connect to SQLite and load DataFrame as SQL table
conn = sqlite3.connect("csce.db")
df.to_sql("tickets", conn, if_exists="replace", index=False)

cursor = conn.cursor()

# --- 1. Escalation Rate by Channel ---
print("🔹 Escalation Rate by Channel:\n")
query1 = """
SELECT
  channel,
  ROUND(100.0 * SUM(CASE WHEN status = 'Escalated' THEN 1 ELSE 0 END) / COUNT(*), 2) AS escalation_rate
FROM tickets
GROUP BY channel;
"""
cursor.execute(query1)
for row in cursor.fetchall():
    print(f"{row[0]}: {row[1]}%")

# --- 2. Average Resolution Time by Agent ---
print("\n🔹 Avg Resolution Time by Agent:\n")
query2 = """
SELECT
  agent_id,
  ROUND(AVG(resolution_time), 2) AS avg_resolution_time
FROM tickets
GROUP BY agent_id;
"""
cursor.execute(query2)
for row in cursor.fetchall():
    print(f"Agent {row[0]}: {row[1]} mins")

# --- 3. Daily Ticket Volume ---
print("\n🔹 Daily Ticket Volume:\n")
query3 = """
SELECT
  ticket_date,
  COUNT(*) AS total_tickets
FROM tickets
GROUP BY ticket_date
ORDER BY ticket_date;
"""
cursor.execute(query3)
for row in cursor.fetchall():
    print(f"{row[0]}: {row[1]} tickets")

# --- 4. Tickets by Status and Channel ---
print("\n🔹 Ticket Status Breakdown by Channel:\n")
query4 = """
SELECT
  channel,
  status,
  COUNT(*) AS total
FROM tickets
GROUP BY channel, status;
"""
cursor.execute(query4)
for row in cursor.fetchall():
    print(f"{row[0]} | {row[1]}: {row[2]}")

# --- 5. Average Satisfaction by Agent ---
print("\n🔹 Average Satisfaction Score by Agent:\n")
query5 = """
SELECT
  agent_id,
  ROUND(AVG(satisfaction_score), 2) AS avg_score
FROM tickets
GROUP BY agent_id;
"""
cursor.execute(query5)
for row in cursor.fetchall():
    print(f"Agent {row[0]}: {row[1]}/5")

# Close connection
conn.close()


🔹 Escalation Rate by Channel:

Chat: 33.33%
Email: 33.33%
Phone: 0.0%

🔹 Avg Resolution Time by Agent:

Agent A1: 68.33 mins
Agent A2: 50.0 mins
Agent A3: 25.0 mins

🔹 Daily Ticket Volume:

2024-06-01: 1 tickets
2024-06-02: 2 tickets
2024-06-03: 1 tickets
2024-06-04: 2 tickets
2024-06-05: 1 tickets
2024-06-06: 1 tickets

🔹 Ticket Status Breakdown by Channel:

Chat | Escalated: 1
Chat | Resolved: 2
Email | Escalated: 1
Email | Resolved: 2
Phone | Resolved: 2

🔹 Average Satisfaction Score by Agent:

Agent A1: 2.33/5
Agent A2: 3.67/5
Agent A3: 5.0/5
